# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [22]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [23]:
stocks = pd.read_csv('sp_500_stocks.csv')


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [24]:
from secrets import IEX_CLOUD_API_TOKEN

symbol = 'AAPL'
base_url = 'https://sandbox.iexapis.com/stable'
api_url = f"{base_url}/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

{'companyName': 'Apple Inc', 'marketcap': 2535802333143, 'week52high': 157.59, 'week52low': 102.94, 'week52highSplitAdjustOnly': 157.28, 'week52lowSplitAdjustOnly': 106.2, 'week52change': 0.20656159510985353, 'sharesOutstanding': 16694775453, 'float': 0, 'avg10Volume': 80706675, 'avg30Volume': 74451949, 'day200MovingAvg': 137.64, 'day50MovingAvg': 147.22, 'employees': 150086, 'ttmEPS': 5.2, 'ttmDividendRate': 0.8795054281283267, 'dividendYield': 0.005879040668999262, 'nextDividendDate': '', 'exDividendDate': '2021-07-27', 'nextEarningsDate': '2021-10-26', 'peRatio': 29.260461024909645, 'beta': 1.52183577005022, 'maxChangePercent': 57.99361313228496, 'year5ChangePercent': 4.882754496063145, 'year2ChangePercent': 1.8918451611998703, 'year1ChangePercent': 0.27061885241753236, 'ytdChangePercent': 0.12451485101764075, 'month6ChangePercent': 0.14851054880742426, 'month3ChangePercent': 0.1665492978491785, 'month1ChangePercent': 0.015779263716810644, 'day30ChangePercent': 0.021043918986554007,

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [25]:
data['year1ChangePercent']

0.27061885241753236

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [26]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

# for symbol_string in symbol_strings:
#     print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [27]:
def momentum_data():
    dataframe = pd.DataFrame(columns = my_columns)

    for symbols in symbol_strings:
        # print(symbols)
        api_url = f"{base_url}/stock/market/batch?symbols={symbols}&types=stats,price&token={IEX_CLOUD_API_TOKEN}"
        data = requests.get(api_url).json()
        # print(data['AAPL'])
        # print(data['AAPL']['stats'])

        for symbol in symbols.split(','):
            price = data[symbol]['price']
            # year1ChangePercent = data[symbol]['stats']['year1ChangePercent'],
            buy_count = 'N/A'

            dataframe = dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        data[symbol]['stats']['year1ChangePercent'],
                        buy_count
                    ],
                    index = my_columns
                ),
                ignore_index = True
            )

    return dataframe

final_dataframe = momentum_data()
print(final_dataframe)

    Ticker   Price One-Year Price Return Number of Shares to Buy
0        A  175.76               0.76542                     N/A
1      AAL   18.50                  0.49                     N/A
2      AAP  212.27               0.35025                     N/A
3     AAPL  152.61              0.273571                     N/A
4     ABBV  121.88              0.314196                     N/A
..     ...     ...                   ...                     ...
500    YUM  137.41              0.447697                     N/A
501    ZBH  150.06              0.073379                     N/A
502   ZBRA  578.90              1.069614                     N/A
503   ZION   54.84              0.738541                     N/A
504    ZTS  208.62              0.322451                     N/A

[505 rows x 4 columns]


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [28]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50] # Dump all but the first 50
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,81.40,2.38949,N/A
1,272,KSS,59.36,2.058045,N/A
2,441,TPR,41.55,1.87713,N/A
3,344,NUE,120.46,1.688173,N/A
4,106,COF,173.43,1.678364,N/A
5,129,DFS,130.39,1.635694,N/A
6,253,IVZ,24.60,1.50669,N/A
7,23,ALB,230.15,1.500494,N/A
8,148,DVN,26.00,1.492363,N/A
9,453,UAA,23.59,1.448602,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [29]:
def portfolio_input():
    global portfolio_size

    portfolio_size = 1000000 # input('Enter the value of your portfolio:')

    try:
        float(portfolio_size)
    except ValueError:
        print('Portfolio size should be an integer.')
        portfolio_size = input('Enter the value of your portfolio:')
        float(portfolio_size)

portfolio_input()
print(portfolio_size)

1000000


In [30]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    buy_count =  math.floor(position_size / final_dataframe.loc[i, 'Price'])
    final_dataframe.loc[i, 'Number of Shares to Buy'] = buy_count

final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,81.40,2.38949,245
1,272,KSS,59.36,2.058045,336
2,441,TPR,41.55,1.87713,481
3,344,NUE,120.46,1.688173,166
4,106,COF,173.43,1.678364,115
5,129,DFS,130.39,1.635694,153
6,253,IVZ,24.60,1.50669,813
7,23,ALB,230.15,1.500494,86
8,148,DVN,26.00,1.492363,769
9,453,UAA,23.59,1.448602,847


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [31]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
]

def hq_momentum_data():
    dataframe = pd.DataFrame(columns = hqm_columns)

    for symbols in symbol_strings:
        # print(symbols)
        api_url = f"{base_url}/stock/market/batch?symbols={symbols}&types=stats,price&token={IEX_CLOUD_API_TOKEN}"
        data = requests.get(api_url).json()
        # print(data['AAPL'])
        # print(data['AAPL']['stats'])

        for symbol in symbols.split(','):
            price = data[symbol]['price']
            # year1ChangePercent = data[symbol]['stats']['year1ChangePercent'],
            buy_count = np.nan

            dataframe = dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        buy_count,
                        data[symbol]['stats']['year1ChangePercent'], 
                        np.nan,
                        data[symbol]['stats']['month6ChangePercent'], 
                        np.nan,
                        data[symbol]['stats']['month3ChangePercent'], 
                        np.nan,
                        data[symbol]['stats']['month1ChangePercent'], 
                        np.nan,
                    ],
                    index = hqm_columns,
                    
                ),
                ignore_index = True
            )

    # https://stackoverflow.com/questions/65174575/typeerror-not-supported-between-instances-of-nonetype-and-float
    return dataframe.fillna(0)

hqm_dataframe = hq_momentum_data()
hqm_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,One-Month Price Return,One-Month Return Percentile,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile
0,A,174.26,0.738652,0.0,0.132407,0.0,0.0,0.329206,0.0,0.276999,0.0
1,AAL,19.40,0.500000,0.0,-0.101692,0.0,0.0,-0.009872,0.0,-0.183599,0.0
2,AAP,213.43,0.360840,0.0,-0.002721,0.0,0.0,0.273641,0.0,0.072831,0.0
3,AAPL,148.88,0.265255,0.0,0.015910,0.0,0.0,0.150648,0.0,0.169113,0.0
4,ABBV,120.96,0.310691,0.0,0.024297,0.0,0.0,0.158598,0.0,0.026635,0.0
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.97,0.454019,0.0,0.159820,0.0,0.0,0.291713,0.0,0.125780,0.0
501,ZBH,150.86,0.070737,0.0,-0.065677,0.0,0.0,-0.072128,0.0,-0.132922,0.0
502,ZBRA,598.34,1.098532,0.0,0.104148,0.0,0.0,0.168064,0.0,0.158324,0.0
503,ZION,56.12,0.747947,0.0,0.083648,0.0,0.0,0.055574,0.0,-0.050296,0.0


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [32]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month',
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        column_return = f'{time_period} Price Return'
        column_percentile = f'{time_period} Return Percentile'

        # print(hqm_dataframe.convert_dtypes().dtypes)
        # print(hqm_dataframe[column_return])
        price_return_all = pd.to_numeric(hqm_dataframe[column_return].astype(float))
        price_return_single = hqm_dataframe.loc[row, column_return]

        percentile = score(price_return_all, price_return_single)
        hqm_dataframe.loc[row, column_percentile] = percentile

hqm_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,One-Month Price Return,One-Month Return Percentile,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile
0,A,174.26,0.738652,0.0,0.132407,94.257426,84.554455,0.329206,87.326733,0.276999,95.643564
1,AAL,19.40,0.500000,0.0,-0.101692,4.356436,65.346535,-0.009872,17.821782,-0.183599,4.752475
2,AAP,213.43,0.360840,0.0,-0.002721,30.891089,46.336634,0.273641,79.405941,0.072831,66.336634
3,AAPL,148.88,0.265255,0.0,0.015910,42.970297,34.257426,0.150648,54.851485,0.169113,86.138614
4,ABBV,120.96,0.310691,0.0,0.024297,47.128713,40.990099,0.158598,56.831683,0.026635,51.881188
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,138.97,0.454019,0.0,0.159820,96.633663,61.188119,0.291713,81.980198,0.125780,78.217822
501,ZBH,150.86,0.070737,0.0,-0.065677,11.683168,13.267327,-0.072128,9.702970,-0.132922,9.900990
502,ZBRA,598.34,1.098532,0.0,0.104148,88.118812,95.643564,0.168064,59.207921,0.158324,84.158416
503,ZION,56.12,0.747947,0.0,0.083648,83.168317,85.742574,0.055574,33.069307,-0.050296,29.108911


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [33]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

NameError: name 'writer' is not defined

## Saving Our Excel Output

As before, saving our Excel output is very easy: